# Day 1: The Tyranny of the Rocket Equation

In [1]:
import os
import unittest

In [2]:
filename = os.path.join(os.getcwd(),"d1_input")
with open(filename) as f:
    mass_inputs = [int(mass) for mass in f.readlines()]

In [3]:
def requiredFuel(mass):
    return mass // 3 -2

def totalFuel(mass):
    input_fuel = requiredFuel(mass)
    f = requiredFuel(input_fuel)
    total_fuel = input_fuel
    while f>0:
        total_fuel += f
        f = requiredFuel(f)
    return total_fuel

def fuelSum(mass_inputs,callback):
    return sum(map(callback,mass_inputs))

In [4]:
# Answer 1
fuelSum(mass_inputs,requiredFuel)

3405637

In [5]:
# Answer 2
fuelSum(mass_inputs,totalFuel)

5105597

# Day 2: 1202 Program Alarm

In [54]:
def run_program(intcode_program):
    for i in range(0,len(intcode_program),4):
        if intcode_program[i]==1:
            intcode_program[intcode_program[i+3]] = intcode_program[intcode_program[i+1]] + intcode_program[intcode_program[i+2]]
        elif intcode_program[i]==2:
            intcode_program[intcode_program[i+3]] = intcode_program[intcode_program[i+1]] * intcode_program[intcode_program[i+2]]
        elif intcode_program[i]==99:
            break
        else:
            raise ValueError(f"Something went wrong. Opcode is {intcode_program[i]} at index {i} for the current program {intcode_program}")
    return intcode_program


In [55]:
class Test(unittest.TestCase):
    def test(self):
        self.states = [([1,9,10,3,2,3,11,0,99,30,40,50],[3500,9,10,70,2,3,11,0,99,30,40,50]),
                ([1,0,0,0,99],[2,0,0,0,99]),
                ([2,3,0,3,99],[2,3,0,6,99]),
                ([2,4,4,5,99,0],[2,4,4,5,99,9801]),
                ([1,1,1,4,99,5,6,0,99],[30,1,1,4,2,5,6,0,99])]
        for (program,output) in self.states:
            self.assertEqual(run_program(program),output)

unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [56]:
filename = os.path.join(os.getcwd(),"d2_input")
with open(filename) as f:
    initial_program = [int(opcode) for opcode in f.read().split(",")]
    intcode_program = initial_program[:]
intcode_program[1]=12
intcode_program[2]=2

# Answer 1
run_program(intcode_program)[0]

False

In [58]:
def findInput(init_program):
    for noun in range(50):
        for verb in range(50):
            program = init_program[:]
            program[1]=noun
            program[2]=verb
            if run_program(program)[0] == 19690720:
                return noun,verb,100*noun+verb
    
    return None

# Answer 2
findInput(initial_program)

(40, 19, 4019)

# Day 3: Crossed Wires

In [26]:
import scipy.sparse as sparse
import numpy as np

In [33]:
filename = os.path.join(os.getcwd(),"d3_input")
wires = []
with open(filename) as f:
    str_wires = f.read().splitlines()

In [54]:
class Point:
    def __init__(self,i,j):
        self.i = i
        self.j = j

class Wire:
    def __init__(self,string):
        self.full_path = [(el[0],int(el[1:])) for el in string.replace("\n","").split(",")]
        self.size = 30#00
        self.central_port = Point(self.size//2,self.size//2)
        # self.sparse = sparse.bsr_matrix((self.size,self.size))
        
        self.create_path()

    def create_path(self):
    
        self.sparse = np.zeros((self.size,self.size))
        def fill_mat():
            current_point = self.central_port
            for (direction,nb_steps) in self.full_path:
                if direction=="U":
                    self.sparse[current_point.i-nb_steps:current_point.i,current_point.j]=1
                    current_point.i-=nb_steps
                elif direction=="D":
                    self.sparse[current_point.i:current_point.i+nb_steps,current_point.j]=1
                    current_point.i+=nb_steps
                elif direction=="L":
                    self.sparse[current_point.i,current_point.j-nb_steps:current_point.j]=1
                    current_point.j+=nb_steps            
                elif direction=="R":
                    self.sparse[current_point.i,current_point.j:current_point.j+nb_steps]=1
                    current_point.j+=nb_steps
        try:
            fill_mat()
        except IndexError as e:
            self.size = int(self.size*1.2)
            print(f"{e}  Change to size {self.size}")
            self.sparse = np.zeros((self.size,self.size))
            fill(mat)
    
    def __str__(self):
        return str(self.sparse)

In [55]:
str_wires = ["R8,U5,L5,D3","U7,R6,D4,L4"]
wires = [Wire(wire) for wire in str_wires]
print(wires[0])

IndexError: index 23 is out of bounds for axis 1 with size 20